In [2]:
import os
import imagehash
import random
import shutil
from PIL import Image
from pathlib import Path

In [3]:
""" GLOBAL VARIABLES """
DATA_DIR: Path = Path("C:/capstone/dataset_experiment")
EXTENSIONS: list[str] = [".png", ".jpg", ".jpeg", ".bmp", ".gif"]
# WHERE_IS_YOUR_DATASET: str = '...'

# Data Cleaning

## CHECK DUPLICATES FOR EACH CLASS

This only check duplicates for each class assuming you provide DATA_DIR path to a folder consisting subfolders, example: 
```
DATA_DIR         
├───subfolder_1 
│   ├───class_1 
│   ├───...     
│   └───class_n 
└───subfolder_n 
    ├───class_1 
    ├─── ...    
    └───class_n 
```
This is tipically what's available on Kaggle

In [4]:
from imagehash import ImageHash


def print_log(duplicates_log: dict[str, list[tuple[str, str]]], 
                   total_duplicates: dict[str, int]) -> None:
    """ Print duplicate images and counts per subfolder """
    if duplicates_log:
        print("\nDuplicate images found:")
        for class_name, duplicate_pairs in duplicates_log.items():
            print(f"\nClass: {class_name}")
            for duplicate, original in duplicate_pairs:
                print(f"Duplicate: {duplicate} | Original: {original}")
        
        print("\nTotal duplicates found per subfolder:")
        for subfolder, count in total_duplicates.items():
            print(f"{subfolder}: {count}")
    else:
        print("No duplicates found in log; \
              Duplicates log not found")
        
def check_structure(where_is_your_ds: str) -> None:
    if 'dataset' not in os.listdir():
        _path = os.path.join('dataset', 'raw')
        os.mkdir(_path)
        if os.path.exists(where_is_your_ds):
            shutil.move(where_is_your_ds, _path)
    
def check_for_dupes(ds_dir: Path):
    """ Check for duplicate images in the dataset """
    duplicates_log: dict[str, list[tuple[str, str]]] = {} # ini simpen Path duplicatenya
    total_duplicates: dict[str, int] = {subfolder.name: 0 for subfolder \
                                        in ds_dir.iterdir() if subfolder.is_dir()}

    for subfolder in ds_dir.iterdir():
        """ Iterate through subfolders (1st layer) """
        subfolder_path: Path = ds_dir / subfolder.name
        print(f"In subfolder {subfolder}:")
        
        for class_folder in subfolder_path.iterdir():
            """ Iterate through class folders (2nd layer) """
            if class_folder.is_dir():
                print(f"Processing {class_folder.name}")
                
                hashes: dict[ImageHash, str] = {} # initialize to store hashes for each class
                
                for image_file in class_folder.iterdir():
                    """ Iterate through image files (3rd; last layer) """
                    if image_file.suffix.lower() in EXTENSIONS:
                        try:
                            with Image.open(image_file) as img:
                                img_hash = imagehash.phash(img) # calculate hash based on perceptual hash
                            
                            if img_hash in hashes: # add duplicates in each class to log
                                duplicates_log.setdefault(class_folder.name, []) \
                                .append((str(image_file), hashes[img_hash]))

                                total_duplicates[subfolder.name] += 1  # Increment total duplicate count
                            else: # add new unique image to hashes
                                hashes[img_hash] = str(image_file)
                        except Exception as e:
                            print(f"Error processing {image_file}: {e}")
    print_log(duplicates_log, total_duplicates)


In [5]:
# check_structure(WHERE_IS_YOUR_DATASET)
check_for_dupes(DATA_DIR)

In subfolder C:\capstone\dataset_experiment\garbage_concat:
Processing battery
Processing biological
Processing brown-glass
Processing bulb
Processing cardboard
Processing ceramic
Processing clothes
Processing diaper
Processing drugs
Processing green-glass
Processing leftovers
Processing mask
Processing metal
Processing paper
Processing plastic
Processing shoes
Processing toothbrush
Processing white-glass

Duplicate images found:

Class: battery
Duplicate: C:\capstone\dataset_experiment\garbage_concat\battery\battery (1)100002.png | Original: C:\capstone\dataset_experiment\garbage_concat\battery\battery (1).png
Duplicate: C:\capstone\dataset_experiment\garbage_concat\battery\battery (10)100004.png | Original: C:\capstone\dataset_experiment\garbage_concat\battery\battery (10).png
Duplicate: C:\capstone\dataset_experiment\garbage_concat\battery\battery (11)100016.png | Original: C:\capstone\dataset_experiment\garbage_concat\battery\battery (11).png
Duplicate: C:\capstone\dataset_experime

## DELETE SAID DUPLICATES
You can see kalo the algorithm is sama

In [6]:
def copy_if_not_exist(source: Path, destination: Path) -> None:
    if not Path.exists(destination):
        shutil.copy2(source, destination)

def delete_dupes(ds_dir: Path) -> None:
    """Check for duplicate images in the dataset."""
    duplicates_log: dict[str, list[tuple[str, str]]] = {}
    total_duplicates: dict[str, int] = {subfolder.name: 0 for subfolder \
                                        in ds_dir.iterdir() if subfolder.is_dir()}

    for subfolder in ds_dir.iterdir():
        """ Iterate through subfolders (1st layer) """
        subfolder_path: Path = ds_dir / subfolder.name
        cleaned_sub_path: Path = ds_dir.parent / 'dupe_cleaned' / subfolder.name

        print(f"In subfolder {subfolder}:")
        
        for class_folder in subfolder_path.iterdir():
            """ Iterate through class folders (2nd layer) """
            if class_folder.is_dir():
                print(f"Processing {class_folder.name}")

                cleaned_class_path: Path = cleaned_sub_path / class_folder.name
                if not Path.exists(cleaned_class_path): # make dir for dupe-cleaned ds
                    cleaned_class_path.mkdir(parents=True, exist_ok=False)
                
                hashes: dict[ImageHash, str] = {} # initialize to store hashes for each class
                
                for image_file in class_folder.iterdir():
                    """ Iterate through image files (3rd; last layer) """
                    if image_file.suffix.lower() in EXTENSIONS:
                        try:
                            with Image.open(image_file) as img:
                                img_hash = imagehash.phash(img) # calculate hash based on perceptual hash
                            
                            if img_hash not in hashes: #skip copy if in hash (duplicate)
                                destination = cleaned_class_path / image_file.name
                                hashes[img_hash] = str(image_file)
                                copy_if_not_exist(image_file, destination)

                        except Exception as e:
                            print(f"Error processing {image_file}: {e}")


In [7]:
delete_dupes(DATA_DIR)

In subfolder C:\capstone\dataset_experiment\garbage_concat:
Processing battery
Processing biological
Processing brown-glass
Processing bulb
Processing cardboard
Processing ceramic
Processing clothes
Processing diaper
Processing drugs
Processing green-glass
Processing leftovers
Processing mask
Processing metal
Processing paper
Processing plastic
Processing shoes
Processing toothbrush
Processing white-glass


In [8]:
check_for_dupes(DATA_DIR.parent / 'dupe_cleaned')

In subfolder C:\capstone\dupe_cleaned\garbage_concat:
Processing battery
Processing biological
Processing brown-glass
Processing bulb
Processing cardboard
Processing ceramic
Processing clothes
Processing diaper
Processing drugs
Processing green-glass
Processing leftovers
Processing mask
Processing metal
Processing paper
Processing plastic
Processing shoes
Processing toothbrush
Processing white-glass
No duplicates found in log;               Duplicates log not found


# Splitting

## SPLIT DATASET INTO (TRAIN, VALIDATION AND TEST)

* **WIP: STILL EXPERIMENTAL**

**ONLY USE THIS CODE IF YOU WANT YOUR DATASET LOOKS LIKE:** <br>
```
dataset             
├───subfolder_1     
│   ├───train       
│   ├───test        
│   └───validation  
└───subfolder_n     
    ├───train       
    ├───test        
    └───validation  
```
**Note:** Classes inside your splitted dataset will remain the same <br>
Example: `dataset/subfolder/train/class_n`

In [9]:
""" GLOBAL VARIABLES FOR SPLITTING """
CLEAN_DS_DIR: Path = DATA_DIR.parent / 'dupe_cleaned'
SPLIT_RATIOS: dict[str, float] = {'train': 0.7, 'validation': 0.2, 'test': 0.1}

In [12]:
def copy_if_not_exist(source: Path, destination: Path) -> None:
    if not Path.exists(destination):
        shutil.copy2(source, destination)

def split_files(files: list[Path]) -> dict[str, list[Path]]:
    """ This function splits the dataset using the list index """
    random.seed(42)
    random.shuffle(files)
    train_i_end: int = int(len(files) * SPLIT_RATIOS['train'])
    val_i_end: int = train_i_end + int(len(files) * SPLIT_RATIOS['validation'])
    return {
        'train': files[:train_i_end],
        'validation': files[train_i_end:val_i_end],
        'test': files[val_i_end:]
    }

def split_dataset(clean_ds_dir: Path) -> None:
    for subfolder in clean_ds_dir.iterdir():
        """ Iterate through subfolders (1st layer) """
        subfolder_path: Path = CLEAN_DS_DIR / subfolder.name
        split_path: Path = CLEAN_DS_DIR.parent / 'split'

        print(f"In subfolder {subfolder}:")
        for class_folder in subfolder_path.iterdir():
            """ Iterate through class folders (2nd layer) """
            if class_folder.is_dir():
                print(f"Processing {class_folder.name}")
                
                files = [ file for file in class_folder.iterdir() # iterate through files in 
                         if file.suffix.lower() in EXTENSIONS ]   # class folders;
                files = split_files(files)                        # store and split files
                
                for split, files in files.items():
                    destination_path: Path = split_path / subfolder.name / split /class_folder.name
                    if not Path.exists(destination_path):
                        destination_path.mkdir(parents=True, exist_ok=False)
                    for file_path in files:
                        destination = destination_path / file_path.name
                        copy_if_not_exist(file_path, destination)


In [13]:
split_dataset(CLEAN_DS_DIR)

In subfolder C:\capstone\dupe_cleaned\garbage_concat:
Processing battery
Processing biological
Processing brown-glass
Processing bulb
Processing cardboard
Processing ceramic
Processing clothes
Processing diaper
Processing drugs
Processing green-glass
Processing leftovers
Processing mask
Processing metal
Processing paper
Processing plastic
Processing shoes
Processing toothbrush
Processing white-glass


# CHECK TRAIN, TEST, VALIDATION FOR DUPES
* **WIP: STILL EXPERIMENTAL**
* **MANUAL**

In [ ]:
# def check_for_data_leakage():
#     # Dictionary to store image hashes and their associated file paths and dataset type
#     hash_dict = {}
#     # Dictionary to store leaks found between datasets
#     data_leakage_log = {subfolder: [] for subfolder in SUB_DIRS}

#     # Iterate through each folder in train, validation, and test
#     for subfolder in SUB_DIRS:
#         subfolder_path = DATA_DIR / subfolder
#         print(f"Checking for data leakage in '{subfolder}' set...")

#         # Iterate through each class folder
#         for class_folder in subfolder_path.iterdir():
#             if class_folder.is_dir():
#                 # Iterate through images in the class folder
#                 for image_file in class_folder.iterdir():
#                     if image_file.suffix.lower() in EXTENSIONS:
#                         try:
#                             # Open image and calculate hash
#                             with Image.open(image_file) as img:
#                                 img_hash = imagehash.phash(img)

#                             # Check if this hash already exists in another dataset
#                             if img_hash in hash_dict:
#                                 # Check if it came from a different subset
#                                 if hash_dict[img_hash]['subfolder'] != subfolder:
#                                     data_leakage_log[subfolder].append((image_file, hash_dict[img_hash]['path']))
#                                     print(f"Duplicates: {image_file} (in '{subfolder}') | {hash_dict[img_hash]['path']} (in '{hash_dict[img_hash]['subfolder']}')")
#                             else:
#                                 # If the hash is unique so far, store it with the file path and subfolder
#                                 hash_dict[img_hash] = {'path': image_file, 'subfolder': subfolder}
#                         except Exception as e:
#                             print(f"Error processing {image_file}: {e}")

#     # # Output data leakage log if any leaks are found
#     # if any(data_leakage_log[subfolder] for subfolder in SUB_DIRS):
#     #     print("\nData leakage detected across datasets:")
#     #     for subfolder, leakage_pairs in data_leakage_log.items():
#     #         if leakage_pairs:
#     #             print(f"\nLeakage in '{subfolder}' set:")
#     #             for duplicate, original in leakage_pairs:
#     #                 print(f"Duplicate in {duplicate} matches with {original}")
#     # else:
#     #     print("No data leakage found between train, validation, and test sets.")

# # Execute the function
# check_for_data_leakage()

Checking for data leakage in 'test' set...
Checking for data leakage in 'train' set...
Duplicates: C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_dupe_cleaned_set\train\aluminum_food_cans\Image_103.png (in 'train') | C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_dupe_cleaned_set\test\steel_food_cans\Image_113.png (in 'test')
Duplicates: C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_dupe_cleaned_set\train\aluminum_food_cans\Image_115.png (in 'train') | C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_dupe_cleaned_set\test\steel_food_cans\Image_104.png (in 'test')
Duplicates: C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_dupe_cleaned_set\train\aluminum_food_cans\Image_6.png (in 'train') | C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_dupe_cleaned_set\test\steel_food_cans\Image_5.png (in 'test')
Duplicates: C:\Users\Keny\Downloads\virtualenv\dataset\cleaned\images_default_du